# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

### Completed and Submitted by Stephen Fox
### June 2016

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
This is a classification problem. We are attempting to classify students into two distinct groups, namely those who need early intervention, since they are likely to fail, and those who do not, since they are likely to pass. A regression problem would involve something like trying to assign a numerical score to each student, which we are not doing here.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [86]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [87]:
# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
# 1 is substracted from the number of columns since the last column is the data label (pass or fail)
n_features = len(student_data.T)-1

# TODO: Calculate passing students
n_passed = student_data.passed.value_counts()['yes']

# TODO: Calculate failing students
n_failed = student_data.passed.value_counts()['no']

# TODO: Calculate graduation rate
grad_rate = (100 * n_passed) / (n_passed + n_failed)

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.00%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [88]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [89]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [90]:
# TODO: Import any additional functionality you may need here
from sklearn.utils import shuffle

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above

X_all, y_all = shuffle(X_all, y_all, random_state=1)

X_train, X_test = X_all[:num_train], X_all[num_train:]

y_train, y_test = y_all[:num_train], y_all[num_train:]

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: **
The learning models need to be capable of handling classification problems. I have chosen the following three models:

1. Decision Trees (DT)
2. Naive Bayes (NB)
3. Support Vector Machines (SVM)

Decision trees are a logic based algorithm and are generally applicable to problems with discrete or categorical features (which appears to be the case with the student features data, by and large), since the tree structure is based on learning simple decision rules from the features. Strengths of DTs are the speed of training and classification, relatively high tolerance to missing or irrelevant attributes and the relative ease with which the model and the model parameters can be understood and intrepreted. Weaknesses of DTs include relatively low accuracies compared to other models, a tendency to overfit and the possibility of instability, since small variations in the data might result in a completely different tree being generated. 

Naive Bayes is a probabilistic based algorithm and has proven to be generally applicable to document classification and spam filtering problems. Strengths of NBs are that they require relatively small data sets and are extremely fast learners and classifiers. Like DTs, they are also relatively easy to understand and interpret, especially for people accustomed to thinking in terms of probabilities. Weaknesses include low accuracy in broader applications (i.e. beyond document classification) and low toleranace to highly interdependent attributes.

Support vector machines are generally applicable to learning problems with continuous attributes (not discrete ones). Strengths of SVM include relatively high accuracy and a high tolerance to irrelevant, redundant or interdependent attributes. Weaknesses of SVM include the requirement for large data sets, slow learning speed, risk of overfitting and models that lack intuitive understanding, making them hard to understand, interpret or explain to others (i.e. they are very "black box" in nature).

I chose these three models because they are quite different from each other. No single algorithm can uniformly outperform other algorithms over all datasets, so I felt that I would like to use three distinct methods to initially get a feel for which one would work best. I know that the data set includes many discrete / categorical features, which would tend to favor DTs. However, I also know that I am dealing with a relatively small training set (300 samples), which plays to one of NBs strengths. However, both DT and NB tend to have low accuracies in many applications, and hence I wanted to utilize a model that is known for generally having high accuracy, namely SVM.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [91]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [95]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# TODO: Initialize the three models
clf_A = DecisionTreeClassifier()
clf_B = GaussianNB()
clf_C = SVC()

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size

train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
print "\n"
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
print "\n"
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)
print "\n"

train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
print "\n"
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
print "\n"
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)
print "\n"

train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
print "\n"
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
print "\n"
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)
print "\n"

Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0011 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.7018.


Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0013 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7097.


Training a DecisionTreeClassifier using a training set size of 300. . .
Trained model in 0.0017 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0001 seconds.
F1 score for test set: 0.6880.


Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0008 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.1842.
Made predictions in 0.0004 seconds.
F1 score for test set: 0.4048.


Training a GaussianNB using a traini

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Decision Tree**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0011           |         0.0003         |       1.00       |     0.7018      |
| 200               |        0.0013           |         0.0002         |       1.00       |     0.7097      |
| 300               |        0.0017           |         0.0002         |       1.00       |     0.6880      |

** Classifer 2 - Naive Bayes**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0008           |         0.0004         |      0.1842      |     0.4048      |
| 200               |        0.0008           |         0.0003         |      0.8129      |     0.7619      |
| 300               |        0.0010           |         0.0003         |      0.8029      |     0.7619      |

** Classifer 3 - Support Vector Machines**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0010           |         0.0006         |      0.8903      |     0.7867      |
| 200               |        0.0033           |         0.0011         |      0.8829      |     0.7838      |
| 300               |        0.0060           |         0.0015         |      0.8745      |     0.7973      |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

I have tested three substantially different machine learning models here and ultimately have chosen to use SVM, since it provides the most accurate results across various training set sizes. The F1 score (a measure of model precision and recall) for the 300 training size test case was 0.80 for SVM, compared to 0.69 and 0.76 for DT and NB, respectively. This difference is too large to overlook and hence I am recommending using SVM going forward.

There were several reasons to think a DT or NB approach might work well with this data set, namely the discrete nature of the student features and the relatively small dataset.  However, these two models failed to give good enough scoring results relative to the SVM results to justify using them over SVM. SVM generally has a much slower speed of learning relative to the other two models, and that was the case here, with SVM on the full training dataset requiring many factors longer than the other two models (e.g. 0.0060s to train vs. 0.0017s and 0.0010s for DT and NB, respectively). But in this case, processing time doesn't appear to be a key issue, given that the SVM model is capable of learning in low millisecond timeframes. So even a significant increase in the available data is unlikely to result in resource or performance issues if SVM is used, given that this model is not being used for an application where differences in milliseconds (or even seconds) are likely to matter. Although the small dataset is a potential problem with SVM in general, this algorithm still scored better than the other models, and as more student data is obtained over time, the SVM approach should only get stronger.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: **

SVM algorithms revolve around the concept of a 'margin' (think of a 'margin of error' in classic design terms as an analogy), which is the gap that separates instances of the class in question. In this particular application, the two instances are whether a student is going to 'pass' or 'fail'. The objective is to maximize this gap, or margin, between the two instances. The algorithm does so by finding 'hyperplanes' (imagine sheets of glass) that can separate these two instances, based on the values of the student features (e.g. gender, age, family size etc.) being measured.

To create the model, the student data provided was divided into a large training set and a small test set. The purpose of the training test is to establish the parameters behind the model. The purpose of the test set is to test how accurately the model performs when it sees new data. Once these margins have been found by the algorithm using training data, then the algorithm is fed test data. Based on training, the model has identified which features are important in a given case (namely those cases where the feature in question lies close to one of these separating margins). The model can then compare the features of the test data and decide on which side of the margins the test case in question lies, and thus classify the test student as 'pass' or 'fail'. 

To use the model for the purposes of intervention, the 30 student features that you measure can be fed into the model for a given student. The model will then predict whether that student is likely to 'pass' or 'fail', based on historical precedent. The school can then provide intervention support to those students deemed likely to fail, thereby changing the course of their outcomes. 

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [96]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune

C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)
parameters = dict(gamma=gamma_range, C=C_range)

# TODO: Initialize the classifier
clf = SVC()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

print("The best parameters are %s with a score of %0.2f"
      % (grid_obj.best_params_, grid_obj.best_score_))

Made predictions in 0.0039 seconds.
Tuned model has a training F1 score of 0.8299.
Made predictions in 0.0014 seconds.
Tuned model has a testing F1 score of 0.8000.
The best parameters are {'C': 100000.0, 'gamma': 9.9999999999999995e-08} with a score of 0.83


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

The results compare as follows:

F1 Training: 0.8745 (untuned) vs. 0.8299 (tuned)
F1 Testing: 0.7973 (untuned) vs. 0.8000 (tuned)

So, the models training score got worse with tuning, but the testing score increased, reaching a level not seen in either the 100, 200 or 300 training set size scenarios for any of the models tested. Since the test score is ultimately what matters, this seems like a good tradeoff, and likely an indicator that the tuned model has struck a good balance and is not overfitting.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.